# MuJoCo SO101 数据采集

这个 Notebook 使用 MuJoCo 仿真环境采集训练数据，用于 SmolVLA 模型训练。

## 🎮 控制说明

**游戏手柄控制**:
- **左摇杆**: X-Y 平面移动
- **右摇杆**: Z 轴移动和旋转
- **触发器**: 夹爪控制
- **Start 按钮**: 结束当前 episode
- **Select 按钮**: 重新录制当前 episode

**键盘控制** (如果使用 KeyboardGamepad 任务):
- **WASD**: X-Y 平面移动
- **RF**: Z 轴上下
- **方向键**: 旋转
- **空格**: 夹爪
- **ESC**: 结束 episode

## 💡 提示

- 每个 episode 应该包含完整的抓取-放置过程
- 尽量保持动作流畅自然
- 录制至少 50 个 episodes 以获得好的训练效果
- 可以通过改变物体位置增加数据多样性

---
## 📦 步骤 0: 导入库

In [ ]:
import json
import logging
from pathlib import Path

import gymnasium as gym
import torch

from lerobot.rl.gym_manipulator import (
    make_robot_env, 
    make_processors, 
    control_loop, 
    GymManipulatorConfig
)
from lerobot.envs.configs import HILSerlRobotEnvConfig
from lerobot.rl.gym_manipulator import DatasetConfig

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ 库导入成功！")

---
## ⚙️ 步骤 1: 配置参数

In [ ]:
# ========== 配置参数 ==========

# 数据集配置
DATASET_REPO_ID = "your_username/mujoco_so101_pickplace"  # 修改为您的 HF 用户名
TASK_DESCRIPTION = "Pick up the cube and place it on the target."
NUM_EPISODES = 50  # 要录制的 episodes 数量
EPISODE_TIME_S = 30.0  # 每个 episode 的最大时长（秒）

# 环境配置
FPS = 20  # 控制频率
TASK_NAME = "PandaPickCubeGamepad-v0"  # 或 "PandaPickCubeKeyboard-v0"

# 输出配置
OUTPUT_DIR = None  # None 表示使用默认路径
PUSH_TO_HUB = False  # 是否上传到 Hugging Face Hub

# 设备
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print("✅ 配置参数设置完成")
print(f"  数据集: {DATASET_REPO_ID}")
print(f"  任务: {TASK_DESCRIPTION}")
print(f"  Episodes: {NUM_EPISODES}")
print(f"  设备: {DEVICE}")

---
## 🔍 步骤 2: 检查依赖

In [ ]:
print("🔍 检查依赖...")

try:
    import gym_hil
    print("✓ gym_hil 已安装")
except ImportError:
    print("✗ gym_hil 未安装!")
    print("请运行: pip install -e '.[hilserl]'")
    raise

# 检查 CUDA
if DEVICE == 'cuda':
    gpu_name = torch.cuda.get_device_name(0)
    print(f"✓ GPU: {gpu_name}")
else:
    print("⚠️ 使用 CPU (数据处理会较慢)")

print()

---
## 📝 步骤 3: 构建配置对象

In [ ]:
# 构建环境配置
config_dict = {
    "env": {
        "type": "gym_manipulator",
        "name": "gym_hil",
        "task": TASK_NAME,
        "fps": FPS,
        "robot": None,
        "teleop": None,
        "processor": {
            "control_mode": "gamepad",
            "gripper": {
                "use_gripper": True,
                "gripper_penalty": -0.02
            },
            "max_gripper_pos": 2.0,
            "reset": {
                "control_time_s": EPISODE_TIME_S,
                "terminate_on_success": True,
                "fixed_reset_joint_positions": [0.0, 0.195, 0.0, -2.43, 0.0, 2.62, 0.785]
            },
            "observation": {
                "display_cameras": True,
                "add_joint_velocity_to_observation": False,
                "add_current_to_observation": False
            },
            "image_preprocessing": None,
            "inverse_kinematics": {
                "urdf_path": None,
                "target_frame_name": "panda_hand",
                "end_effector_step_sizes": {
                    "x": 0.025,
                    "y": 0.025,
                    "z": 0.025
                },
                "end_effector_bounds": {
                    "x": [-0.5, 0.5],
                    "y": [-0.5, 0.5],
                    "z": [0.0, 0.8]
                }
            },
            "reward_classifier": None
        }
    },
    "dataset": {
        "repo_id": DATASET_REPO_ID,
        "root": OUTPUT_DIR,
        "task": TASK_DESCRIPTION,
        "num_episodes_to_record": NUM_EPISODES,
        "replay_episode": None,
        "push_to_hub": PUSH_TO_HUB
    },
    "mode": "record",
    "device": DEVICE
}

# 构建配置对象
env_config = HILSerlRobotEnvConfig(**config_dict['env'])
dataset_config = DatasetConfig(**config_dict['dataset'])

gym_config = GymManipulatorConfig(
    env=env_config,
    dataset=dataset_config,
    mode=config_dict['mode'],
    device=config_dict['device']
)

print("✅ 配置对象创建成功")

---
## 📋 步骤 4: 打印配置信息

In [ ]:
print("=" * 70)
print("🎮 MuJoCo SO101 数据采集")
print("=" * 70)
print()
print("📋 配置信息:")
print(f"  环境: {gym_config.env.task}")
print(f"  FPS: {gym_config.env.fps}")
print(f"  数据集名称: {gym_config.dataset.repo_id}")
print(f"  任务描述: {gym_config.dataset.task}")
print(f"  Episodes 数量: {gym_config.dataset.num_episodes_to_record}")
print(f"  Episode 时长: {gym_config.env.processor.reset.control_time_s}s")
print(f"  保存路径: {gym_config.dataset.root or '默认'}")
print(f"  上传到 Hub: {gym_config.dataset.push_to_hub}")
print(f"  设备: {gym_config.device}")
print("=" * 70)
print()

---
## 🚀 步骤 5: 创建环境和处理器

In [ ]:
print("🚀 初始化环境...")

# 创建环境
env, teleop_device = make_robot_env(gym_config.env)
print("✓ 环境创建成功")

# 创建处理器
env_processor, action_processor = make_processors(
    env, teleop_device, gym_config.env, gym_config.device
)
print("✓ 处理器创建成功")

print()
print(f"观测空间: {env.observation_space}")
print(f"动作空间: {env.action_space}")
print()

---
## 📝 步骤 6: 使用说明

### 开始前检查清单

- ✓ 游戏手柄已连接（或使用键盘）
- ✓ 配置参数已正确设置
- ✓ 确认数据集名称正确

### 录制技巧

1. **每个 episode 包含完整任务**: 从起始位置 → 抓取物体 → 放置到目标位置
2. **动作流畅自然**: 避免突然的移动或卡顿
3. **增加多样性**: 改变物体位置、抓取角度
4. **质量优于数量**: 50 个高质量 episodes 比 100 个低质量的更好

### 控制器映射

**游戏手柄**:
- 左摇杆 → 机械臂 X-Y 移动
- 右摇杆 → Z 轴和旋转
- 触发器 → 夹爪开合
- Start → 完成当前 episode
- Select → 重新录制

准备好后，运行下一个单元格开始录制！

---
## 🎬 步骤 7: 开始数据采集

In [ ]:
print("\n🎬 开始数据采集...\n")
print("控制说明:")
print("  - 使用游戏手柄控制机械臂")
print("  - 完成一个抓取-放置任务后，按 Start 结束 episode")
print("  - 如果失败，按 Select 重新录制当前 episode")
print("  - 按 Ctrl+C 可以中断录制")
print()

try:
    # 开始数据采集
    control_loop(env, env_processor, action_processor, teleop_device, gym_config)
    
    print("\n✅ 数据采集完成!")
    
    if gym_config.dataset.push_to_hub:
        print(f"📤 数据已上传到: https://huggingface.co/datasets/{gym_config.dataset.repo_id}")
    else:
        print(f"💾 数据已保存到本地: {gym_config.dataset.root or '默认路径'}")
        
except KeyboardInterrupt:
    print("\n⚠️ 用户中断数据采集")
except Exception as e:
    print(f"\n❌ 发生错误: {e}")
    raise
finally:
    env.close()
    print("环境已关闭")

---
## 📊 步骤 8: 验证数据集（可选）

In [ ]:
# 如果想立即查看采集的数据，可以运行这个单元格

from lerobot.datasets.lerobot_dataset import LeRobotDataset

print("📂 加载数据集...")

try:
    # 从本地或 Hub 加载
    dataset = LeRobotDataset(
        DATASET_REPO_ID,
        root=OUTPUT_DIR
    )
    
    print(f"✅ 数据集加载成功")
    print(f"  Episodes: {dataset.num_episodes}")
    print(f"  Frames: {dataset.num_frames}")
    print(f"  FPS: {dataset.fps}")
    print(f"  特征: {list(dataset.features.keys())}")
    
except Exception as e:
    print(f"❌ 无法加载数据集: {e}")
    print("数据集可能还在处理中，请稍后再试")

---
## ✅ 数据采集完成！

### 下一步

1. **检查数据质量**:
   - 至少有 50 个高质量 episodes
   - 每个 episode 包含完整任务
   - 动作流畅自然

2. **继续训练**:
   - 打开 `02_train_smolvla.ipynb`
   - 使用采集的数据训练 SmolVLA 模型

3. **可视化数据** (可选):
   - 在 Hugging Face 上查看: https://huggingface.co/spaces/lerobot/visualize_dataset
   - 输入您的数据集 repo_id

### 数据质量建议

**推荐做法**:
- 50+ episodes
- 5 个不同的物体位置
- 每个位置 10 个演示
- 包含足够的变化以提高泛化能力

---

**🎉 祝您数据采集顺利！**